In [1]:
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill, Border, Alignment, Font, Protection
from copy import copy

def copiar_aba_openpyxl(arquivo_origem, arquivo_destino, nome_aba, max_linhas=5000, max_colunas=40):
    wb_origem = load_workbook(arquivo_origem, data_only=False)
    ws_origem = wb_origem[nome_aba]

    wb_destino = Workbook()
    ws_destino = wb_destino.active
    ws_destino.title = nome_aba

    # Copiando dados e estilo com limites
    for row in ws_origem.iter_rows(min_row=1, max_row=max_linhas, min_col=1, max_col=max_colunas):
        for cell in row:
            nova_celula = ws_destino.cell(row=cell.row, column=cell.column, value=cell.value)
            if cell.has_style:
                nova_celula.font = copy(cell.font)
                nova_celula.border = copy(cell.border)
                nova_celula.fill = copy(cell.fill)
                nova_celula.number_format = copy(cell.number_format)
                nova_celula.protection = copy(cell.protection)
                nova_celula.alignment = copy(cell.alignment)

    # Copiar largura das colunas (até o limite)
    for col_idx in range(1, max_colunas + 1):
        col_letter = ws_origem.cell(row=1, column=col_idx).column_letter
        if col_letter in ws_origem.column_dimensions:
            ws_destino.column_dimensions[col_letter].width = ws_origem.column_dimensions[col_letter].width

    # Copiar altura das linhas (até o limite)
    for row_idx in range(1, max_linhas + 1):
        if row_idx in ws_origem.row_dimensions:
            ws_destino.row_dimensions[row_idx].height = ws_origem.row_dimensions[row_idx].height

    wb_destino.save(arquivo_destino)


In [4]:
origem = r'C:\dev\Itau\Superveniência\superveniencia_piscofins\Input\Anexo C\Dashboard.xlsx'
nome_aba = "PIS_COFINS_ANUAL"
destino = r"C:\dev\Itau\Superveniência\superveniencia_piscofins\Output\arquivo_novo.xlsx"

In [29]:

# Exemplo de uso
# Exemplo de uso:
copiar_aba_openpyxl(origem, destino, "PIS_COFINS_ANUAL")


In [30]:
from openpyxl import load_workbook

def limpar_celulas_com_textos_ou_apagar_linha(arquivo, nome_aba, termos_limpar=None, termos_apagar_linha=None, salvar_como=None):
    wb = load_workbook(arquivo)
    ws = wb[nome_aba]

    termos_limpar = termos_limpar or []
    termos_apagar_linha = termos_apagar_linha or []

    linhas_para_apagar = set()

    for row in ws.iter_rows(min_row=3):
        for cell in row:
            if cell.value is not None:
                cell_val = str(cell.value).strip()

                # 👉 Apagar linha só se for EXATAMENTE igual ao termo
                if any(cell_val == term for term in termos_apagar_linha):
                    linhas_para_apagar.add(cell.row)

                # 👉 Limpar célula se contiver algum termo
                elif any(cell_val == term for term in termos_limpar):
                    cell.value = None

    for row_idx in sorted(linhas_para_apagar, reverse=True):
        ws.delete_rows(row_idx)

    if salvar_como:
        wb.save(salvar_como)
    else:
        wb.save(arquivo)


In [27]:
termos = ["NumContrato", "Pesquisa Número Contrato:", "PIS", "(Multiple Items)", "Soma de Mov_Formatado", "Column Labels", "4572449", "Exclusão", "Dedução", "PIS-COFINS - Dedução", "Sem efeito"]
termos_linha = ["Row Labels"]

In [31]:
limpar_celulas_com_textos_ou_apagar_linha(destino, nome_aba, termos, termos_linha)
